# Get Area-Elevation Curve

In [1]:
import ee
ee.Initialize(project='tmospp')

In [2]:
# from rat.ee_utils.ee_aec_file_creator import aec_file_creator
import geopandas as gpd
from pathlib import Path
import hvplot.pandas
import pandas as pd
import holoviews as hv
import geoviews as gv
import numpy as np

hv.extension('bokeh')

### select the reservoir

In [3]:
start_date = '2023-07-21'
end_date = '2024-10-30'
RESERVOIR = '0810'

ALG_TYPE = 'swot'
DATA_DIR = Path('../data')

In [4]:
# read the bounding box of the study area
val_pts = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-pts.geojson'))
val_polys = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-poly.geojson'))

selected_reservoirs = val_pts['tmsos_id'].tolist()
res_names = val_pts[['tmsos_id', 'name']].set_index('tmsos_id').to_dict()['name']

RESERVOIR_NAME = res_names[RESERVOIR]
print(f'{RESERVOIR}: {RESERVOIR_NAME}')

val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

nominal_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_SKM'].values[0]
nominal_area_poly = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_POLY'].values[0]
max_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MAX'].values[0]
max_area = np.nan if max_area == -99 else max_area
min_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MIN'].values[0]
min_area = 0 if min_area == -99 else min_area
area_rep = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_REP'].values[0]
dam_height = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DAM_HGT_M'].values[0])
elev_msl = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['ELEV_MASL'].values[0])
depth = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DEPTH_M'].values[0])
capacity = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['CAP_MCM'].values[0])

global_map = (
    val_res_pt.hvplot(
        geo=True, tiles='OSM'
    ) * val_res_pt[val_res_pt['tmsos_id'] == RESERVOIR].hvplot(
        geo=True, color='red', size=100, 
    )
).opts(
    title=f"Locations of validation reservoirs. {RESERVOIR_NAME}, highlighted in red"
)

global_map

0810: Noi, Th


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]
   .Points.II :Points   [Longitude,Latitude]

In [5]:
val_polys['db'] = val_polys['db'].where(val_polys['rise_id'].isna(), 'rise')

## Storage Calculation

In [6]:
# what is the reported capacity?
capacity_hv = hv.HLine(capacity).opts(color='red', ylim=(0, capacity + capacity*0.1), ylabel='capacity (Mil. m3)')
capacity_hv

:HLine   [x,y]

In [8]:
aec_dir = Path('../data/aec/aev_insitu')

In [9]:
val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

aec_fp = aec_dir / f'{RESERVOIR}.csv'
aec_df = pd.read_csv(aec_fp, comment='#')

aec_df

,Elevation,Storage (mil. m3)
0,137.00,794.222391
1,137.25,839.823400
2,137.50,886.814552
3,137.75,935.195847
4,138.00,984.967286
5,138.25,1036.128868
6,138.50,1088.680593
7,138.75,1142.622461
8,139.00,1197.954472
9,139.25,1254.676627


# Calculate storage using insitu aev.

In [17]:
import xarray as xr
import hvplot.xarray
import numpy as np


# inp_dir = Path('/tiger1/pdas47/tmsosPP/data/storage/icesat2/2324-insitu')
inp_dir = Path('/tiger1/pdas47/tmsosPP/data/elevation/swot_karin')
fp = inp_dir / f'{RESERVOIR}.csv'

data = {
    'elevation': [],
    'date': [],
}

df = pd.read_csv(fp, parse_dates=['time'])
data['elevation'] = df['elevation']
data['date'] = df['time']
data['storage'] = np.interp(data['elevation'], aec_df['Elevation'], aec_df['Storage (mil. m3)'])

reservoir_dynamics = pd.DataFrame(data).set_index('date')

reservoir_dynamics.reset_index()

,date,elevation,storage
0,2023-07-31,NaN,NaN
1,2023-08-21,NaN,NaN
2,2023-08-31,NaN,NaN
3,2023-09-04,141.14912,1731.122889
4,2023-09-11,147.61766,2043.773650
5,2023-09-21,141.02185,1696.585222
6,2023-10-02,142.60382,2043.773650
7,2023-10-12,NaN,NaN
8,2023-10-15,142.83560,2043.773650
9,2023-10-23,142.60146,2043.773650


In [18]:
print(capacity)
(capacity_hv * (reservoir_dynamics).hvplot.scatter(x='date', y='storage').opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME}. Storage (mil. m3)', ylabel='Storage', xlabel='Date'
))

1966.0


:Overlay
   .HLine.I   :HLine   [x,y]
   .Scatter.I :Scatter   [date]   (storage)